# Molecular properties
## Structure optimization

One of the most commonly calculated properties in theoretical chemistry is the molecular structure. It is quite common to use structures calculated at a cheaper level of theory, typically DFT, as a starting point since ground state energies are relatively insensitive to the geometry. However, in some cases, especially when strong correlation is significant, one may want to use MCSCF structures instead.

The workflow here is very similar to that of VeloxChem, namely first defining the Gradient Driver, and then give it to the optimizer, which itself is based on GeomeTRIC. The MCSCF gradient driver will automatically choose analytical gradients if available for the specific type of MCSCF calculation performed, numerical gradients otherwise.

```{note}
Currently, derivative integrals are not available and all gradients are therefore numerical. This should change very soon.
```

Let's illustrate this on a simple O$_2$ molecule. First, we compute one MCSCF with the $\pi$ orbitals in the active space.

In [1]:
import veloxchem as vlx
import multipsi as mtp

O2_xyz="""2

O 0.0 0.0 -0.6
O 0.0 0.0  0.6
"""
molecule=vlx.Molecule.from_xyz_string(O2_xyz)
molecule.set_multiplicity(1)
basis = vlx.MolecularBasis.read(molecule,"cc-pvdz")

scfdrv = vlx.ScfUnrestrictedDriver()
scfdrv.compute(molecule, basis)
uhf_orbs = scfdrv.natural_orbitals()

space=mtp.OrbSpace(molecule,uhf_orbs)
space.cas(6,4)
mcscfdrv=mtp.McscfDriver()
mc_results = mcscfdrv.compute(molecule,basis,space)

* Warning * Environment variable OMP_NUM_THREADS not set.
* Warning * Setting OMP_NUM_THREADS to 8.
* Info * Reading basis set from file: /opt/anaconda3/envs/vlxenv/lib/python3.9/site-packages/veloxchem/basis/CC-PVDZ      
                                                                                                                          
                                              Molecular Basis (Atomic Basis)                                              
                                                                                                                          
                               Basis: CC-PVDZ                                                                             
                                                                                                                          
                               Atom Contracted GTOs           Primitive GTOs                                              
                                       

                                                                                                                          
                          Multi-Configurational Self-Consistent Field Driver
                                                                                                                          
        ╭────────────────────────────────────╮
        │          Driver settings           │
        ╰────────────────────────────────────╯
          State-specific calculation
          Max. iterations         : 50
          BFGS window             : 5
          Convergence thresholds:
            - Energy change       : 1e-08
            - Gradient norm       : 0.0001
                                                                                                                          
          Integrals in memory
                                                                                                                          

          Active space def

Now, we optimize the molecule.

In [2]:
GradDrv=mtp.McscfGradientDriver(mcscfdrv)
opt_drv = vlx.OptimizationDriver(GradDrv)
casscf_opt_geom = opt_drv.compute(molecule, basis, space)

                                                                                                                          
                                                Optimization Driver Setup                                                 
                                                                                                                          
                                     Coordinate System       :    TRIC                                                    
                                     Constraints             :    No                                                      
                                     Max. Number of Steps    :    300                                                     
                                     Transition State        :    No                                                      
                                     Hessian                 :    never                                                   
                

* Info *   Energy   : -149.6134612443 a.u.                                                                                
* Info *   Gradient : 1.499815e-03 a.u. (RMS)                                                                             
* Info *              1.500021e-03 a.u. (Max)                                                                             
* Info *   Time     : 2.48 sec                                                                                            
                                                                                                                          
* Info * Computing energy and gradient...                                                                                 
                                                                                                                          
                          Multi-Configurational Self-Consistent Field Driver
                                                              

* Info *   Energy   : -149.6134625768 a.u.                                                                                
* Info *   Gradient : 3.545858e-04 a.u. (RMS)                                                                             
* Info *              3.547886e-04 a.u. (Max)                                                                             
* Info *   Time     : 1.99 sec                                                                                            
                                                                                                                          
* Info * Computing energy and gradient...                                                                                 
                                                                                                                          
                          Multi-Configurational Self-Consistent Field Driver
                                                              

* Info *   Energy   : -149.6134626552 a.u.                                                                                
* Info *   Gradient : 1.606049e-06 a.u. (RMS)                                                                             
* Info *              1.781861e-06 a.u. (Max)                                                                             
* Info *   Time     : 2.24 sec                                                                                            
                                                                                                                          
* Info * Saving file: veloxchem_opt_2023-06-05T14.08.13_optim.xyz                                                         
* Info * Saving file: veloxchem_opt_2023-06-05T14.08.13.log                                                               
                                                                                                                          
* Info * Geometr

We can now use this new geometry in subsequent calculations.

In [3]:
molecule=casscf_opt_geom
mc_results = mcscfdrv.compute(molecule,basis,space)

                                                                                                                          
                          Multi-Configurational Self-Consistent Field Driver
                                                                                                                          
        ╭────────────────────────────────────╮
        │          Driver settings           │
        ╰────────────────────────────────────╯
          State-specific calculation
          Max. iterations         : 50
          BFGS window             : 5
          Convergence thresholds:
            - Energy change       : 1e-08
            - Gradient norm       : 0.0001
                                                                                                                          
          Integrals in memory
                                                                                                                          

          Active space def

If your MCSCF calculation is a state-averaged, you should extend the parameters of the compute function with the number of states in the state-averaging and the index (starting from 0) of the state you wish to optimize:

```
casscf_opt_geom = opt_drv.compute(molecule, basis, space, nstates, istate)
```

## Spectroscopy

### State-averaging

MCSCF is commonly used to simulate spectroscopy and photochemistry. There are several ways to compute excited state properties from a MCSCF calculation. As discussed in the previous section, one way is to use state-averaging:

Let's go back to our furan calculation. First we compute a SA-CASSCF with 5 states:

In [4]:
furan_xyz="""9

 C     -0.86213    -0.90784     0.00007
 H     -1.63433    -1.64264    -0.00003
 C      0.50727    -0.90524     0.00007
 C      0.92057     0.47886    -0.00003
 C     -0.22323     1.23186    -0.00003
 O     -1.35123     0.40376    -0.00013
 H      1.17117    -1.74724     0.00017
 H      1.93767     0.81866     0.00007
 H     -0.46573     2.26986    -0.00013
"""

molecule = vlx.Molecule.from_xyz_string(furan_xyz)
basis = vlx.MolecularBasis.read(molecule,"def2-sv(p)")

#We use a previously stored h5 file for the active space
space=mtp.OrbSpace(molecule,"furan-cas.h5")
mcscfdrv=mtp.McscfDriver()
mc_results = mcscfdrv.compute(molecule,basis,space, 5) #state-averaged with 5 states

* Info * Reading basis set from file: /opt/anaconda3/envs/vlxenv/lib/python3.9/site-packages/veloxchem/basis/DEF2-SV_P_   
                                                                                                                          
                                              Molecular Basis (Atomic Basis)                                              
                                                                                                                          
                               Basis: DEF2-SV(P)                                                                          
                                                                                                                          
                               Atom Contracted GTOs           Primitive GTOs                                              
                                                                                                                          
                

Now we can compute the transition properties using the InterState module. This module requires the dictionary returned by the MCSCF.

In [5]:
#Compute oscillator strengths
SI=mtp.InterState()
Prop_dict=SI.compute(molecule,basis, mc_results)

                                                                                                                          
List of oscillator strengths greather than 1e-10
                                                                                                                          
  From     to       Energy (eV)    Oscillator strength (length and velocity)
     1       2        6.66708         3.935228e-03    2.925456e-03
     1       3        8.29230         2.426600e-01    1.672872e-01
     1       5       10.11827         5.706685e-01    4.028088e-01
                                                                                                                          
List of rotatory strengths greater than 1e-10
                                                                                                                          
  From     to       Energy (eV)    Rot. strength (a.u. and 10^-40 cgs)
     1       2        6.66708         8.842626e-08        0.0000

The function returns a dictionary with the most important values.

In [6]:
print(Prop_dict["energies"])
print(Prop_dict["oscillator_strengths"])

[0.24501075 0.30473646 0.35263678 0.37183941]
[3.93522815e-03 2.42660014e-01 3.30510323e-25 5.70668499e-01]


By default InterState computes properties from the first state to all others, but this can be changed by providing the list of "initial" and "final" states as arguments.

In [7]:
Prop_dict=SI.compute(molecule,basis,mc_results, from_states=[0,1], to_states='all' )

                                                                                                                          
List of oscillator strengths greather than 1e-10
                                                                                                                          
  From     to       Energy (eV)    Oscillator strength (length and velocity)
     1       2        6.66708         3.935228e-03    2.925456e-03
     1       3        8.29230         2.426600e-01    1.672872e-01
     1       5       10.11827         5.706685e-01    4.028088e-01
     2       3        1.62522         2.610885e-02    2.178061e-02
     2       5        3.45118         1.364278e-03    1.798930e-04
                                                                                                                          
List of rotatory strengths greater than 1e-10
                                                                                                                          
 

### Linear response

Another way to compute excited state properties at the MCSCF level is using linear response. This multiconfigurational linear response (MCLR) requires first a MCSCF calculation of the ground state and then the use of the Mclr_EigenSolver module:

In [8]:
mcscfdrv=mtp.McscfDriver()
mc_results = mcscfdrv.compute(molecule,basis,space) #only ground state!

mcrpa = mtp.Mclr_EigenSolver()
Prop_dict = mcrpa.compute(molecule, basis, mcscfdrv, n_states = 5)

                                                                                                                          
                          Multi-Configurational Self-Consistent Field Driver
                                                                                                                          
        ╭────────────────────────────────────╮
        │          Driver settings           │
        ╰────────────────────────────────────╯
          State-specific calculation
          Max. iterations         : 50
          BFGS window             : 5
          Convergence thresholds:
            - Energy change       : 1e-08
            - Gradient norm       : 0.0001
                                                                                                                          
          Integrals in memory
                                                                                                                          

          Active space def

                                                                                                                          
Initialization time: 00:00:00
                                                                                                                          
          Linear eigensolver
          ------------------
                                                                                                                          
CI parameters:      54
Orbital parameters: 1235
                                                                                                                          
                                                                                                                          
                                                                                                                          
        MC-RPA Iterations
        -----------------
                                                                          

Again, the Prop_dict dictionary gives us access to the most important values. It is also possible to get an analysis of the dominant excitation character using the command:

In [9]:
mcrpa.get_excitation_character()

                                                                                                                          
    Excitation characters
    ---------------------
                                                                                                                          
* State 1
- Exc. energy: 0.24232834425546068
 Electrons     Excitation
 0.344    :   17  ->   19
 0.260    :   18  ->   20
 CI weight: 0.971  Orb weight: 0.029
                                                                                                                          
* State 2
- Exc. energy: 0.27129090032428
 Electrons     Excitation
 0.906    :   18  ->   19
 CI weight: 0.928  Orb weight: 0.072
                                                                                                                          
* State 3
- Exc. energy: 0.325775479389993
 Electrons     Excitation
 0.303    :   18  ->   20
 0.264    :   17  ->   19
 0.241    :   16  ->   19
 CI weight: 0.924 

In this analysis, we can see one of the most attractive feature of linear response compared to state-averaging: linear response also includes excitation from or to non-active orbitals. In this example, state 4 and 5 include excitations from the $\pi$ orbitals to some higher lying $\sigma$ orbitals. This feature is particularily useful for molecules with heteroatom lone pairs which do not contribute to the correlation in the active space (and are often difficult to keep in it) but often contribute to some of the lowest-lying excitations.

Another advantage of linear response is that unlike state-averaging, the excitation energies does not depend on the number of calculated states (except when the code accidentally converges to a higher solution, in which case increasing the number of states tends to ensure we get actually the lowest ones). Overall, in most cases, linear response provides more accurate excitation energies than state-averaging.

Note that the Tamm-Dancoff approximation is also available (using the property "mcrpa.tda = True") which gives a slightly faster but less accurate calculation.